In [1]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = '1'

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline

import IPython.display as ipd
import pickle as pkl
import librosa
from text import *
import numpy as np
import torch
import hparams
from modules.model import Model
from text import phonemes_to_sequence
from g2p_en import G2p
from text.cleaners import punctuation_removers


In [2]:
g2p = G2p()
def prepare_input(text):
    text = punctuation_removers(text)
    phonemes = g2p(text)
    #print(phonemes)
    for i in range(0,len(phonemes)):
        if phonemes[i] == " ":
            phonemes[i]= "pau"

    sequence = phonemes_to_sequence(phonemes)
    return sequence
    
    

In [3]:
seq = prepare_input("Most of the local authorities embarked into considerable expenditure, determined to rebuild their jails de novo on the most approved pattern")

In [27]:
seq = prepare_input("Hello my name is john")

In [35]:
for step in ['80000']:
    print(f"checkpoint_{step}")
    checkpoint_path = f"training_log/fastspeech2-phone/checkpoint_{step}"
    state_dict = {}
    for k, v in torch.load(checkpoint_path)['state_dict'].items():
        state_dict[k[7:]]=v
    model = Model(hparams)
    model.load_state_dict(state_dict)
    _ = model.cuda().eval()
    sequence = torch.autograd.Variable(torch.from_numpy(seq)).cuda().long()
    with torch.no_grad():
        melspec, durations, p, e = model.inference(sequence, alpha=1.0)
    print(melspec.shape)
    break

checkpoint_80000
torch.Size([1, 19])
torch.Size([19, 1, 256])
torch.Size([19, 1, 256]) output of Encoder
torch.Size([1, 19]) shape of durations
226 Number of mel frames
torch.Size([226, 1, 256]) Shape of expanded after LR
torch.Size([226, 1, 256]) hidden states expanded
torch.Size([1, 226]) P S
torch.Size([1, 226]) E S
torch.Size([226, 1, 256]) hidden states expanded after e and p encoding
torch.Size([226, 1, 256]) torch.Size([1, 226, 256]) torch.Size([1, 226, 256])
torch.Size([226, 1, 256]) hidden states expanded after positional encoding
torch.Size([1, 226]) Shape of mel Mask
torch.Size([226, 1, 256])
torch.Size([1, 80, 226])


In [36]:
melspec.shape

torch.Size([1, 80, 226])

In [37]:
vocoder = torch.hub.load('seungwonpark/melgan', 'melgan')

Using cache found in C:\Users\Karan's/.cache\torch\hub\seungwonpark_melgan_master


In [39]:
vocoder.eval()
if torch.cuda.is_available():
    vocoder = vocoder.cuda()
    melspec = melspec.cuda()

with torch.no_grad():
    audio = vocoder.inference(melspec)

In [40]:
import IPython
import IPython.display as ipd

In [41]:
ipd.Audio(audio.cpu().float().numpy(), rate=22050) # load a NumPy array